In [1]:
#Packages
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import sys
# sys.path.insert(0, './miniconda3/envs/hali/lib/python3.11/site-packages')
from datasets import load_dataset

/home/hali/miniconda3/envs/hali/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Download data
train_set = load_dataset("ms_marco", "v1.1", split="train")
val_set = load_dataset("ms_marco", "v1.1", split="validation")
test_set = load_dataset("ms_marco", "v1.1", split="test") 

In [3]:
train = train_set.to_pandas()
val = val_set.to_pandas()
test = test_set.to_pandas()

In [4]:
train.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [ ]:
train.to_csv("train.pa",index=False)

In [ ]:
ex = pd.read_csv("train.csv")

In [ ]:
ex.head()

In [ ]:
val.head()

In [ ]:
val.to_csv("val.csv",index=False)

In [ ]:
test.head()

In [ ]:
test.to_csv("test.csv",index=False)

# Making Sentence Tokenizer

In [ ]:
#Making txt file of sentences in answers, passages, and query columns in train dataset
#first queries
corpus = list(train["query"].values)

print(corpus[:3])

In [ ]:
#store passages as one list
passages = list(np.concatenate(train["passages"].apply(lambda x: x["passage_text"])))

In [ ]:
len(passages)

In [ ]:
passages[:3]

In [ ]:
#add to corpus
corpus.extend(passages)

In [ ]:
len(corpus)

In [ ]:
list(np.concatenate(train["answers"]))

In [ ]:
#Add answers to corpus
answers = list(np.concatenate(train["answers"]))
corpus.extend(answers)

In [ ]:
len(corpus)

In [ ]:
#make a new text file with the ms_marco sentences
f = open("ms_marco.txt", "x")
for line in corpus:
    f.write(f"{line}\n")
f.close()

In [ ]:
#Training new sentence piece tokenizer
import sentencepiece as spm

# Input text file for training
input_file = "/home/hali/halimlx/Week2/ms_marco.txt"

# Output model prefix
model_prefix = "/home/hali/halimlx/Week2/ms_spm_model"

# Vocabulary size (adjust as needed)
vocab_size = 32000

# Train SentencePiece model
spm.SentencePieceTrainer.train(
    f"--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size}"
)

In [ ]:
# Load SentencePiece model
model_path = "/home/hali/halimlx/Week2/ms_spm_model.model"
sp = spm.SentencePieceProcessor()
sp.load(model_path)

In [ ]:
sentence = "This is a sample sentence."
tokens = sp.encode(sentence)
print(tokens)

# Make new csv files for train, val and test sets


In [5]:
train.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [6]:
new_train = train[["query","passages"]]

In [7]:
new_train.head()

,query,passages
0,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]..."
1,was ronald reagan a democrat,"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag..."
2,how long do you need for sydney and surroundin...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]..."
3,price to install tile in shower,"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '..."
4,why conversion observed in body,"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas..."


In [10]:
new_val= val[["query","passages"]]

In [11]:
new_test = test[["query","passages"]]

In [8]:
import random

In [9]:
def get_positive_examples(row):
    pos_exs = list(row["passages"].get("passage_text"))
    return pos_exs

In [12]:
new_train["pos_exs"] = new_train.apply(get_positive_examples,axis=1)

/tmp/ipykernel_3087792/1355259439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train["pos_exs"] = new_train.apply(get_positive_examples,axis=1)


In [13]:
new_train.head()

,query,passages,pos_exs
0,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ..."
1,was ronald reagan a democrat,"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...","[In his younger years, Ronald Reagan was a mem..."
2,how long do you need for sydney and surroundin...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...","[Sydney, New South Wales, Australia is located..."
3,price to install tile in shower,"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...","[In regards to tile installation costs, consum..."
4,why conversion observed in body,"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...","[Conclusions: In adult body CT, dose to an org..."


In [15]:
len(new_train["pos_exs"][0])

10

In [16]:
new_val["pos_exs"] = new_val.apply(get_positive_examples,axis=1)
new_test["pos_exs"] = new_test.apply(get_positive_examples,axis=1)

/tmp/ipykernel_3087792/3152730291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_val["pos_exs"] = new_val.apply(get_positive_examples,axis=1)
/tmp/ipykernel_3087792/3152730291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test["pos_exs"] = new_test.apply(get_positive_examples,axis=1)


In [17]:
def get_neg_idx(row,len_df):
    row.name
    neg_idx = row.name
    while neg_idx == row.name:
        neg_idx = random.randint(0, len_df-1)
    return neg_idx

In [18]:
new_train["neg_idx"] = new_train.apply(get_neg_idx,len_df=len(new_train),axis=1)

/tmp/ipykernel_3087792/3577417797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train["neg_idx"] = new_train.apply(get_neg_idx,len_df=len(new_train),axis=1)


In [19]:
new_train.head()

,query,passages,pos_exs,neg_idx
0,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027
1,was ronald reagan a democrat,"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...","[In his younger years, Ronald Reagan was a mem...",51548
2,how long do you need for sydney and surroundin...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...","[Sydney, New South Wales, Australia is located...",38154
3,price to install tile in shower,"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...","[In regards to tile installation costs, consum...",13656
4,why conversion observed in body,"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...","[Conclusions: In adult body CT, dose to an org...",27091


In [20]:
new_val["neg_idx"] = new_val.apply(get_neg_idx,len_df=len(new_val),axis=1)
new_test["neg_idx"] = new_test.apply(get_neg_idx,len_df=len(new_test),axis=1)

/tmp/ipykernel_3087792/277527548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_val["neg_idx"] = new_val.apply(get_neg_idx,len_df=len(new_val),axis=1)
/tmp/ipykernel_3087792/277527548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test["neg_idx"] = new_test.apply(get_neg_idx,len_df=len(new_test),axis=1)


In [21]:
def get_neg_exs(row,df):
    neg_idx = row.neg_idx
    neg_exs = list(df["passages"][neg_idx]["passage_text"])
    return neg_exs

In [22]:
new_train["neg_exs"] = new_train.apply(get_neg_exs,df=new_train,axis=1)

/tmp/ipykernel_3087792/1645880820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train["neg_exs"] = new_train.apply(get_neg_exs,df=new_train,axis=1)


In [23]:
new_train.head()

,query,passages,pos_exs,neg_idx,neg_exs
0,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...
1,was ronald reagan a democrat,"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...","[In his younger years, Ronald Reagan was a mem...",51548,[A committee (or commission) is a type of smal...
2,how long do you need for sydney and surroundin...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...","[Sydney, New South Wales, Australia is located...",38154,"[Robert Louis Fosse was born in Chicago, Illin..."
3,price to install tile in shower,"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...","[In regards to tile installation costs, consum...",13656,[This cultural blend is seen in Paraguay's for...
4,why conversion observed in body,"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...","[Conclusions: In adult body CT, dose to an org...",27091,"[Elephant ivory is the most important source, ..."


In [26]:
new_val["neg_exs"] = new_val.apply(get_neg_exs,df=new_val,axis=1)
new_test["neg_exs"] = new_test.apply(get_neg_exs,df=new_test,axis=1)

/tmp/ipykernel_3087792/4014232044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_val["neg_exs"] = new_val.apply(get_neg_exs,df=new_val,axis=1)
/tmp/ipykernel_3087792/4014232044.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test["neg_exs"] = new_test.apply(get_neg_exs,df=new_test,axis=1)


In [27]:
new_train["examples"]=new_train["pos_exs"]+new_train["neg_exs"]
new_val["examples"]=new_val["pos_exs"]+new_val["neg_exs"]
new_test["examples"]=new_test["pos_exs"]+new_test["neg_exs"]

In [28]:
new_train["examples"]

0        [Since 2007, the RBA's outstanding reputation ...
1        [In his younger years, Ronald Reagan was a mem...
2        [Sydney, New South Wales, Australia is located...
3        [In regards to tile installation costs, consum...
4        [Conclusions: In adult body CT, dose to an org...
                               ...                        
82321    [definition of propagation the act or action o...
82322    [The Path to Becoming a Psychologist. First, y...
82323    [If you need a wine to pair with oysters or mu...
82324    [Your doctor will want to take regular blood t...
82325    [Polarity Index. Burdick & Jackson solvents ar...
Name: examples, Length: 82326, dtype: object

In [29]:
new_train["examples"][0]

["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.",
 "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.",
 'RBA R

In [31]:
def get_labels(row):
    return [1]*len(row.pos_exs)+[0]*len(row["neg_exs"])

In [32]:
new_train["labels"] = new_train.apply(get_labels,axis=1)

In [33]:
new_train.head()

,query,passages,pos_exs,neg_idx,neg_exs,examples,labels
0,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...,"[Since 2007, the RBA's outstanding reputation ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
1,was ronald reagan a democrat,"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...","[In his younger years, Ronald Reagan was a mem...",51548,[A committee (or commission) is a type of smal...,"[In his younger years, Ronald Reagan was a mem...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,how long do you need for sydney and surroundin...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...","[Sydney, New South Wales, Australia is located...",38154,"[Robert Louis Fosse was born in Chicago, Illin...","[Sydney, New South Wales, Australia is located...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]"
3,price to install tile in shower,"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...","[In regards to tile installation costs, consum...",13656,[This cultural blend is seen in Paraguay's for...,"[In regards to tile installation costs, consum...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]"
4,why conversion observed in body,"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...","[Conclusions: In adult body CT, dose to an org...",27091,"[Elephant ivory is the most important source, ...","[Conclusions: In adult body CT, dose to an org...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [34]:
new_val["labels"] = new_val.apply(get_labels,axis=1)
new_test["labels"] = new_test.apply(get_labels,axis=1)

In [55]:
exploded_new_train = new_train.explode(['examples', 'labels']).reset_index(drop=True)

In [56]:
exploded_new_train.head()

,query,passages,pos_exs,neg_idx,neg_exs,examples,labels
0,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...,"Since 2007, the RBA's outstanding reputation h...",1
1,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...,The Reserve Bank of Australia (RBA) came into ...,1
2,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...,RBA Recognized with the 2014 Microsoft US Regi...,1
3,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...,The inner workings of a rebuildable atomizer a...,1
4,what is rba,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[Since 2007, the RBA's outstanding reputation ...",75027,[1 Having a roofing contractor remove (and dis...,Results-Based Accountability® (also known as R...,1


In [58]:
type(new_train["pos_exs"][0])

list

In [59]:
exploded_new_val = new_val.explode(['examples', 'labels']).reset_index(drop=True)
exploded_new_test = new_test.explode(['examples', 'labels']).reset_index(drop=True)

In [63]:
from sklearn.model_selection import train_test_split

In [68]:
train_ds, test_ds = train_test_split(exploded_new_val, test_size=0.2, random_state=1)

train_ds, val_ds,  = train_test_split(train_ds, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (2130399014.py, line 1)

In [72]:
train_ds = train_ds.reset_index(drop=True)
val_ds = val_ds.reset_index(drop=True)
test_ds = test_ds.reset_index(drop=True)

In [73]:
train_ds.head()

,query,passages,pos_exs,neg_idx,neg_exs,examples,labels
0,how much does a general practice doctor make,"{'is_selected': [0, 0, 0, 0, 1], 'passage_text...",[Family Practice Doctor Salary and General Pra...,3520,"[In a commercial real estate, a net lease requ...","In a triple net lease, the tenant is responsib...",0
1,strep throat contagious period,"{'is_selected': [0, 1, 1, 0, 0, 0, 0], 'passag...",[Strep throat is a contagious disease caused b...,899,[tuition and fees for a public four year insti...,The contagious period is over after being on a...,1
2,does florida require malpractice insurance for...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0], 'pas...",[Print Article Email Article. As a nurse pract...,8484,[A nutritionist’s salary varies based on the a...,Nutritionist salary in USA. According to the B...,0
3,average cost to drain pool,"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0], '...",[1 Re-plastering the interior surface of an av...,2312,[Noun. 1. hydrogen bond-a chemical bond consis...,1 Re-plastering the interior surface of an ave...,1
4,source of radioactive gaseous radon in homes,"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0, 0], '...",[Radioactivity. Radioactivity is the spontaneo...,1451,[Hypnosis is a state of human consciousness in...,Hypnosis in Psychology. Throughout the history...,0


In [74]:
train_ds.to_csv("final_train.csv",index=False)
val_ds.to_csv("final_val.csv",index=False)
test_ds.to_csv("final_test.csv",index=False)

In [60]:
# exploded_new_train.to_csv("final_train.csv",index=False)
# exploded_new_val.to_csv("final_val.csv",index=False)
# exploded_new_test.to_csv("final_test.csv",index=False)

KeyboardInterrupt: 

In [62]:
len(exploded_new_val)

164794

In [57]:
exploded_new_train[["query","examples","labels"]].head(21)

,query,examples,labels
0,what is rba,"Since 2007, the RBA's outstanding reputation h...",1
1,what is rba,The Reserve Bank of Australia (RBA) came into ...,1
2,what is rba,RBA Recognized with the 2014 Microsoft US Regi...,1
3,what is rba,The inner workings of a rebuildable atomizer a...,1
4,what is rba,Results-Based Accountability® (also known as R...,1
5,what is rba,Results-Based Accountability® (also known as R...,1
6,what is rba,"RBA uses a data-driven, decision-making proces...",1
7,what is rba,vs. NetIQ Identity Manager. Risk-based authent...,1
8,what is rba,"A rebuildable atomizer (RBA), often referred t...",1
9,what is rba,Get To Know Us. RBA is a digital and technolog...,1
